The `RSA.generate()` method in Python's `pycryptodome` library generates random RSA key pairs (a public key and a corresponding private key). To understand how it avoids generating two identical keys, let's delve into the underlying process.

1. **Random Number Generation**: Generating RSA keys involves generating large prime numbers. These prime numbers are chosen randomly, making it extremely unlikely that the same prime numbers will be chosen twice.

2. **Probabilistic Primality Testing**: The `RSA.generate()` method uses a probabilistic primality test, specifically the Miller-Rabin primality test, to check if a number is prime. This test is not 100% guaranteed to detect primes, but it is very effective and significantly reduces the likelihood of composite numbers being mistaken for primes.

3. **Key Length**: The security of RSA depends on the key length. Longer key lengths provide greater security but also increase the computational overhead. `RSA.generate()` requires specifying the desired key length, such as 2048 bits or 4096 bits. Longer key lengths increase the number of possible key combinations, making it even more improbable to generate identical keys.

4. **Seed Initialization**: The random number generator used by `pycryptodome` is typically seeded with various sources of randomness from the operating system, such as system time, process ID, and other environmental factors. This helps ensure a good initial entropy pool and reduces the likelihood of generating the same key pairs on different runs of the program.

Despite these precautions, it is essential to recognize that generating identical RSA keys is still theoretically possible, albeit extremely improbable. If the random number generator were to be compromised or suffer from inadequate entropy, it could potentially lead to the generation of identical keys.

To mitigate these risks, it is crucial to use a secure and well-established cryptographic library like `pycryptodome`, keep the library and dependencies up-to-date, and ensure a good source of randomness during key generation. Additionally, key pairs should be kept confidential, and key management practices should be followed to minimize any security risks.

In [1]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

#SERIALIZATION CAN ONLY BE USED TO WRITE/READ PRIVATE KEYS TO/FROM FILE, NOT TO SIGN DATA

#SERIALIZE THE ENCRYPTED PRIVATE KEY
private_encrypted_pem = private_key.private_bytes(
   encoding=serialization.Encoding.PEM,
   format=serialization.PrivateFormat.PKCS8,
   encryption_algorithm=serialization.BestAvailableEncryption(b'mypassword')
)


#SERIALIZE THE PRIVATE KEY (WITH NO ENCRYPTION)
private_pem = private_key.private_bytes(
   encoding=serialization.Encoding.PEM,
   format=serialization.PrivateFormat.TraditionalOpenSSL,
   encryption_algorithm=serialization.NoEncryption()
)


#print(private_encrypted_pem.splitlines() == private_pem.splitlines()) == #FALSE


with open("./privateKeyEncrypted.pem", 'wb') as pem_out:
        pem_out.write(private_encrypted_pem)

with open("./privateKeyEncrypted.pem", "rb") as key_file:
    pr_key_encrypted = serialization.load_pem_private_key(
        key_file.read(),
        password=b'mypassword',
    )


with open("./privateKey.pem", 'wb') as pem_out:
        pem_out.write(private_pem)

with open("./privatekey.pem", "rb") as key_file:
    pr_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
    )
    
    
#MY KEYS ARE: pr_key_encrypted & pr_key


In [2]:
pub_key = pr_key.public_key()


#SERIALIZE THE PUBLIC KEY
pem = pub_key.public_bytes(
   encoding=serialization.Encoding.PEM,
   format=serialization.PublicFormat.SubjectPublicKeyInfo
)

#print(pem.splitlines())

#print(pem.decode().splitlines())

author_pub_key = ''

for i in pem.decode().splitlines():
    if (i!='-----BEGIN PUBLIC KEY-----' and i!='-----END PUBLIC KEY-----' ):
        author_pub_key += i
    
print(author_pub_key)
print(len('MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAxJek0uITyn0mzqCsCulOE2HFdkX95C5D6yy47COr7ab77b8gIDjh+SNZvREnynWp3I1nATZEX1o7iIBP18hMkNpPW5/AtH4WcX5PieVWh7ORlL/JR4dtRU2NQRcmOE0R6IMWjEIJ4V9WskdV//6gq1uo1kOajko5+WZxfvxuH5K++x6Jd5fcZxAgWtovByqEZuCrL+iDWdSJwyfwaiikV27ZGTsXf361DVEEU1R1Oebd5jBcibFSYrEk9VCZwAAuFRdy4MBM16ACLpecyoDkWFnyIZXmrb9XqVzWZ0GLFvHqmUXkfT2nHS0OXPr2tuyqzTBrl42qVcKHmHSMdKru3QIDAQAB'))

MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAnSX7OMIrFE4Xgg0g9yl+Wc/82nncMP/MrGSKvslaHIERQMofih5Xjx/AFURLEJNzHjqlPJhNcSdviwtUuA8WYU3/OP28iVjQAURYoQDpd2/GgBr893qd2+ZsGJlYJH1plRKu01rUvACbmojyFxfpC6yq5yeYstatduVV+O+psj4ZXiV2ZV0tk8K8NiVH6b3I3wA/MhH4HZXNIGpoi6MrZNTWBbcox802GOV8HJAK5dZ5/b6gREwog2Z3WQTRXNhRDiC0o+YDlbAXKnwSGmM5HnxWXs+naZmAtyhM+9ko9wDLu8XCf0Nt+D2sSfsd/mvdXu9D1/H+ZA0KPZnfoERp3wIDAQAB
392


In [12]:
file = open("DATA_MESSAGE.txt", "r")
data = file.read()
file.close()
#print(data)
message = str.encode(data)
signature = pr_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

In [17]:
signature

b'A_\x80\xd2N\x02\xbb\xe30@\xdb\xae%\xe4\xce\xee\xc1\x11\xd6S@\x88\x05\x8c\x1c\xff\x9eHe\xdaSWU\x93P\x9b\xaf\xc2D\xcc3\xac\xbd\x92b\xddX)\x96\x17l\xa0\xbc\xd8x\x9f\xcd\x81\xe4\xcd\xa6\xcbW\x1c\xf0\n\x13\x0c\xa6J\xfa\x8bE"\xe7\x92\x9c\x1f\x8e\xca\x18\xd2~\x12l\xa6\x1a\x14V.\x003\xe0\x10\x83\xa1)\xce\xc0\xa4H\xaa\xb7\x91\x15\xbf\xf5\x843\xc66\xd9\x8f\xf2\x97n\xb7$\x8e\xe6-*u\xd3s` \x84\x15<\x8e1\xd7z\xf0\x04q\xd5\x17\xecR]\xc1\xf7`H\x9c+\xe2\xfeB\xb1{\xdc\xf39\xb7\xe9\xb5\xaaR\xce\x99\x08>[\x06\xcc1K\x9b\xe4K%\x82^\xbbv\x8a\xa5YJM\xd3\x94\xa6o\xecyN\xb4\x95\x17\xc2v\xbbk\xb6Q\x1d>\x847g*E\xa7\xd02<C\xce1\xe8\xf6\xe0W]\x84\x142\x86\xf4\xd3\xb0\x94\xae\xda\xf1,`\x9b\'\xcf=\xb9\xd3\xa7\xe6s\xf32\xa7\xd4\x16\xb4\x04\xf0_\x95G%\xaf\xaa\xf9i'

In [23]:
from cryptography.exceptions import InvalidSignature
try:
    outcome = pub_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
except InvalidSignature:
    print('Signature Verification Failed')

Signature Verification Failed


In [12]:
a = 'stringaMatta'
arr = bytes(a, 'utf-8')
print(arr)
arr2 = bytes(a, 'ascii')
print(arr2)

b'stringaMatta'
b'stringaMatta'


In [14]:
my_str = "hello world"
my_str_as_bytes = str.encode(my_str)
print(my_str_as_bytes)
print(type(my_str_as_bytes)) # ensure it is byte representation
my_decoded_str = my_str_as_bytes.decode()
print(my_decoded_str)
print(type(my_decoded_str))

b'hello world'
<class 'bytes'>
hello world
<class 'str'>
